In [1]:
!apt-get update
!apt-get install cmake xvfb python-opengl ffmpeg libgle3 x11-utils

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
E: dpkg was interrupted, you must manually run 'dpkg --configure -a' to correct the problem. 


In [2]:
!pip uninstall tensorflow keras tensorboard tensorflow-gpu -y

Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0
Uninstalling Keras-2.2.5:
  Successfully uninstalled Keras-2.2.5
Uninstalling tensorboard-1.15.0:
  Successfully uninstalled tensorboard-1.15.0


In [3]:
!pip install --upgrade setuptools
!pip install --upgrade tensorflow==1.12 tensorboard keras==2.2.4
!pip install ez_setup gym gym[all] pyvirtualdisplay pyglet==1.3.2 keras-rl

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (45.2.0)
     |████████████████████████████████| 83.1MB 69.7MB/s 
     |████████████████████████████████| 3.9MB 48.7MB/s 
     |████████████████████████████████| 317kB 47.5MB/s 
ERROR: tensorflow 1.12.0 has requirement tensorboard<1.13.0,>=1.12.0, but you'll have tensorboard 2.1.1 which is incompatible.


In [0]:
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

In [0]:
from glob import glob
from io import open as io_open
from base64 import b64encode
from numpy.random import seed

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarn

In [0]:
from gym import logger as gymlogger, make as make_env
from gym.wrappers import Monitor

In [0]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [0]:
def wrap_env(env):
  return Monitor(env, './video', force=True)

In [0]:
#ENV_NAME = 'CartPole-v1'
ENV_NAME = 'MsPacman-v0'
#ENV_NAME = 'Ant-v1'
env = wrap_env(make_env(ENV_NAME))
env.seed(123)
seed(123)
nb_actions = env.action_space.n
gymlogger.set_level(40) # Error only.
display = Display(visible=0, size=(1400, 900))
display.start()
%matplotlib inline

In [0]:
env = wrap_env(env)
def show_video():
    mp4list = glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io_open(mp4, 'r+b').read()
        encoded = b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                                    loop controls style="height: 400px;">
                                    <source src="data:video/mp4;base64,{0}" 
                                    type="video/mp4" /> </video>'''.format(
                                        encoded.decode('ascii'))))
    else: 
        print('Could not find video')

In [12]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 100800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1612816   
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 153       
Total params: 1,613,513
Trainable params: 1,613,513
Non-trainable params: 0
_________________________________________________________________


In [0]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()

In [0]:
# Enable the dueling network (dueling_type is one of {'avg', 'max', 'naive'}).
dqn = DQNAgent(model=model, 
               nb_actions=nb_actions, 
               memory=memory, 
               nb_steps_warmup=30,
               enable_dueling_network=True, 
               dueling_type='avg', 
               target_model_update=0.01, 
               policy=policy)

In [15]:
dqn.compile(Adam(lr=0.001), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)
dqn.save_weights('duel_dqn_' + ENV_NAME + '_weights.h5f', overwrite=True)
dqn.test(env, nb_episodes=10, visualize=True)

Training for 50000 steps ...


/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   645/50000: episode: 1, duration: 41.779s, episode steps: 645, steps per second: 15, episode reward: 220.000, mean reward: 0.341 [0.000, 10.000], mean action: 3.851 [0.000, 8.000], mean observation: 72.876 [0.000, 228.000], loss: 13688.164242, mean_absolute_error: 26.182765, mean_q: 23.794817
  1320/50000: episode: 2, duration: 43.176s, episode steps: 675, steps per second: 16, episode reward: 180.000, mean reward: 0.267 [0.000, 10.000], mean action: 3.901 [0.000, 8.000], mean observation: 72.860 [0.000, 228.000], loss: 1.699579, mean_absolute_error: 0.085622, mean_q: -0.017308
  1793/50000: episode: 3, duration: 29.936s, episode steps: 473, steps per second: 16, episode reward: 220.000, mean reward: 0.465 [0.000, 10.000], mean action: 3.992 [0.000, 8.000], mean observation: 72.846 [0.000, 228.000], loss: 1.551118, mean_absolute_error: 0.553851, mean_q: 0.742153
  2435/50000: episode: 4, duration: 41.359s, episode steps: 642, steps per second: 16, episode reward: 190.000, mean reward

Error: ignored

In [16]:
# Reexecute this cell after crash.
dqn.done = True
show_video()